In [7]:
import torch
import numpy as np
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from DNN_module import Net

%run ../NCoinDP_functions.ipynb
torch.set_default_device("cpu")

In [2]:
a = torch.load("infer_sim_1.pt")
a[0].size()

torch.Size([2500000, 5])

In [ ]:
# Synthetic data for uncertainty
tmp = "infer_sim_1.pt" # uniform prior
tmp = torch.load(tmp)
X = tmp[0]
Y = tmp[1]

# test data, but wanted to extract just a single (first) row
n = 3000
scenarios = ["S1", "S2", "S3"]
test_save_name = '../test_data/OU_test_n'+ str(n) + '_' + scenarios[2] +'.pt'
test = torch.load(test_save_name)
torch.set_default_device("cpu")

# True parameter
print(test[1][1])

x0 = OU_summary(test[0][1])[0,:]
x0 = torch.reshape(x0, (1,5))

uncertainty_list = []
r_list = [.05 ,.07, .10]
for r in r_list:
    for sim in range(3):
        X = X.to("cpu")
        Y = Y.to("cpu")
        x0 = x0.to("cpu")

        L = X.size()[0]
        L_train = 250000
        L_val = 50000
        L_cal = L - L_train - L_val

        # Divide Data
        X_cal = X[(L_train + L_val):].to("cpu")
        Y_cal = Y[(L_train + L_val):].to("cpu")

        model_save_name = str(sim) +'.pt'
        path = F"infer_nets/net1/{model_save_name}"
        torch.set_default_device("cpu")

        # load net
        net = Net(D_in = 5, D_out = 3, H = 256, H2 = 256, H3 = 256)
        net.load_state_dict(torch.load(path))
        net.eval()

        tmp = conf_inf_sd(x0, X_cal, Y_cal, net, r) # .05, .07, .10

        uncertainty_list.append(tmp)
        print("====================", "sim: ", sim, "=====================" )
        torch.save(uncertainty_list, "uncertainty_" +str(r)+"_1.pt") # .05, .07, .10

[3, 2, 1.0]
train_loss: 0.202825  [   64/77000]
train_loss: 0.193393  [ 6464/77000]
train_loss: 0.201696  [12864/77000]
train_loss: 0.191302  [19264/77000]
train_loss: 0.198184  [25664/77000]
train_loss: 0.193752  [32064/77000]
train_loss: 0.174416  [38464/77000]
train_loss: 0.193251  [44864/77000]
train_loss: 0.181549  [51264/77000]
train_loss: 0.179214  [57664/77000]
train_loss: 0.189848  [64064/77000]
train_loss: 0.187561  [70464/77000]
train_loss: 0.184335  [76864/77000]
Epoch 1
-------------------------------
train_loss 0.177986 val_loss 0.178510
Epoch 31
-------------------------------
train_loss 0.020403 val_loss 0.020630
train_loss: 0.022407  [   64/77000]
train_loss: 0.019513  [ 6464/77000]
train_loss: 0.023698  [12864/77000]
train_loss: 0.014880  [19264/77000]
train_loss: 0.018164  [25664/77000]
train_loss: 0.018465  [32064/77000]
train_loss: 0.021665  [38464/77000]
train_loss: 0.021580  [44864/77000]
train_loss: 0.017626  [51264/77000]
train_loss: 0.028470  [57664/77000]
tra

Epoch 31
-------------------------------
train_loss 0.021120 val_loss 0.021044
train_loss: 0.020833  [   64/77000]
train_loss: 0.023791  [ 6464/77000]
train_loss: 0.022642  [12864/77000]
train_loss: 0.021709  [19264/77000]
train_loss: 0.023629  [25664/77000]
train_loss: 0.020698  [32064/77000]
train_loss: 0.025929  [38464/77000]
train_loss: 0.022345  [44864/77000]
train_loss: 0.016671  [51264/77000]
train_loss: 0.021829  [57664/77000]
train_loss: 0.017406  [64064/77000]
train_loss: 0.020672  [70464/77000]
train_loss: 0.021584  [76864/77000]
Epoch 61
-------------------------------
train_loss 0.021073 val_loss 0.021024
Epoch 91
-------------------------------
train_loss 0.021047 val_loss 0.021009
0.02100001
iter:  6 Val=  0.020582462
train_loss: 0.190235  [   64/77000]
train_loss: 0.189228  [ 6464/77000]
train_loss: 0.159948  [12864/77000]
train_loss: 0.179250  [19264/77000]
train_loss: 0.131094  [25664/77000]
train_loss: 0.138754  [32064/77000]
train_loss: 0.138973  [38464/77000]
train

In [4]:
import pandas as pd
tmp = torch.load("uncertainty_10_1.pt")
print(len(tmp))
print(".10")


cvrg_results = np.zeros( (len(tmp),3,3 ))
cvrg_results.shape

# Read a CSV file into a DataFrame
df = pd.read_csv('MCMC/MC_results_1.txt', sep="\t")
MC_results = torch.tensor(df.values)

param_name = ["kappa", "beta", "sigma2"]
coverage = [0.85, 0.90, 0.95]

for k in range(len(tmp)):
    for i in range(3):
        values = tmp[k][1][:,i].detach().cpu().numpy().tolist()
        weights = tmp[k][0].numpy()
        for j in range(3):
            error_cvrg = 1-coverage[j]
            interval = [error_cvrg/2, 1 - error_cvrg/2]
            intvl = weighted_quantile(values, interval, weights)

            cvrg = torch.sum((MC_results[:,i] < intvl[1]) & (intvl[0] < MC_results[:,i]))/MC_results.size()[0]
            
            cvrg_results[k,i,j] = cvrg.detach().cpu().numpy().tolist()
            #print("coverage: ", coverage[j], param_name[i], "inf_m: ", cov_m)
            #print("coverage: ", coverage[j], param_name[i],"inf_sd: ", cvrg)
for i in range(3):
    for j in range(3):
        #print("coverage: ", coverage[j], "param: ", param_name[i], round(np.mean(cvrg_results[:,i,j]),3), "sd: ", round(np.std(cvrg_results[:,i,j]),3))
        x = cvrg_results[:,i,j]
        print("coverage: ", coverage[j], "param: ", param_name[i], round(np.median(x),3), "mad: ", round(np.median(np.absolute(x - np.median(x))),3))

20
.01
coverage:  0.85 param:  kappa 0.819 mad:  0.006
coverage:  0.9 param:  kappa 0.878 mad:  0.007
coverage:  0.95 param:  kappa 0.926 mad:  0.004
coverage:  0.85 param:  beta 0.845 mad:  0.012
coverage:  0.9 param:  beta 0.893 mad:  0.01
coverage:  0.95 param:  beta 0.948 mad:  0.006
coverage:  0.85 param:  sigma2 0.786 mad:  0.055
coverage:  0.9 param:  sigma2 0.855 mad:  0.049
coverage:  0.95 param:  sigma2 0.923 mad:  0.027
